In [ ]:
# Use this script to test and find the best silence lens and threshold
# Splices fragment of audio file

from pydub import AudioSegment
from pydub.utils import make_chunks


audio_path = input("Input audio path") #path refers to the file
loaded_audio = AudioSegment.from_file(audio_path, format="m4a")

predefined_ms = 60 * 1000 # pydub calculates in millisec
spliced_audio = loaded_audio[:predefined_ms] #splices the first seconds of the audio
#spliced_audio_last_ms = loaded_audio[-predefined_ms:] # splices the last seconds of the audio

spliced_audio.export("TEEEEST.wav", format="wav")# Use this script to test and find the best 

In [11]:
import os
from pydub import AudioSegment
from pydub.utils import make_chunks
from pydub.silence import split_on_silence

# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

# Load your audio.
audio_path = input("Input audio path") #path refers to the file
audio_dir = os.path.dirname(audio_path) #dir refers to the folder
loaded_audio = AudioSegment.from_file(audio_path, format="m4a")

#Get the base audio name.
base_name = os.path.basename(audio_path) #outputs 'file.ext'
#base_name_array = os.path.splitext(base_name) #outputs ('file', '.ext')
base_name_wo_ext = os.path.splitext(base_name)[0] #outputs 'file' only

#make a new directory to store exported audio chunks
new_dir =  audio_dir + "/" + base_name_wo_ext
try:
    os.makedirs(new_dir, exist_ok=False) #Will not remake directory if directory exists
    print("New folder sucesfully created in: " + new_dir)
except OSError as error:
    print("File already exists. Making directory skipped")

# Split track where the silence is longer than "min_silence_len" ms
chunks = split_on_silence (
    loaded_audio, 

    min_silence_len = 4 * 1000,

    # anything under the defined dBFS is considered silence
    # the lower the silence threshold, the less sensitive it can pick silence
    silence_thresh = -30

    # keep 200 ms of leading/trailing silence
    #keep_silence=200
)

# target length defines how many seconds
target_length = 5 * 1000
output_chunks = [chunks[0]]
# Process each chunk with your parametersi
for i, chunk in enumerate(chunks):
    # Create a silence chunk that's 0.5 seconds (or 500 ms) long for padding.
    #silence_chunk = AudioSegment.silent(duration=0)

    # Add the padding chunk to beginning and end of the entire chunk.
    #audio_chunk = silence_chunk + chunk + silence_chunk

    # Normalize the entire chunk.
    #normalized_chunk = match_target_amplitude(audio_chunk, -20.0)
    normalized_chunk = match_target_amplitude(chunk, -20.0)
    
    #Skips chunks if audio is under target_length
    if len(output_chunks[-1]) < target_length:
        print('Chunk skipped. It is under target length')

    #Required??, otherwise it will output chunks under target length???????
    elif len(output_chunks[-1]) == target_length:
        print("Audio equals "  + str(target_length) +  "ms. Exporting as: " + base_name_wo_ext + "{0}.wav.".format(i+1))
        normalized_chunk.export(
            new_dir + "/" + base_name_wo_ext + "{0}.wav".format(i+1),
            format = "wav")
        
    #Splices and exports chunk if it over the target_length
    else:
        print("Audio is over "  + str(target_length) +  "ms. Exporting as: " + base_name_wo_ext + "{0}.wav.".format(i+1))
        normalized_chunk.export(
            new_dir + "/" + base_name_wo_ext + "{0}.wav".format(i+1),
            format = "wav")
        

File already exists. Making directory skipped
Audio is over 5000ms. Exporting as: TEEEEST1.wav.
Audio is over 5000ms. Exporting as: TEEEEST2.wav.
Audio is over 5000ms. Exporting as: TEEEEST3.wav.
Audio is over 5000ms. Exporting as: TEEEEST4.wav.
Audio is over 5000ms. Exporting as: TEEEEST5.wav.
Audio is over 5000ms. Exporting as: TEEEEST6.wav.


In [ ]:
import os
from pydub import AudioSegment
from pydub.utils import make_chunks

defined_length_ms = 5 * 1000

for file in os.listdir(new_dir):
     filename = os.fsdecode(file)
     # only picks up files ending with wav.
     if filename.endswith(".wav") or filename.endswith(".mp3"): 
         
        # get file path name for iteration
        audio_path = os.path.join(new_dir, filename) 
        print("Iterating files: " + audio_path)

        #get directory of file - to export files
        audio_dir = os.path.dirname(audio_path)
     
        #Get just the base name of the files
        base_name = os.path.basename(audio_path) #replaces base_name from new dir
        base_name_wo_ext = os.path.splitext(base_name)[0] #replaced base_name_wo_ext with new audio
      
       

        #Assign directory to Audio segment
        loaded_audio = AudioSegment.from_file(audio_path)
        #checks the length of audio
        audio_length_ms = loaded_audio.duration_seconds * 1000 

        if audio_length_ms > defined_length_ms:
            print(base_name_wo_ext + " is longer than " + str(defined_length_ms) + "ms")
            #splices audio files into chunks with the defined length ms
            chunks = make_chunks(loaded_audio, defined_length_ms) 

            #Exports the spliced audio files
            for i, chunk in enumerate(chunks):
                chunk_name = base_name_wo_ext + "-{0}.wav".format(i+1)
                print ("Splicing and exporting as:", chunk_name)
                chunk.export(audio_dir + "/" + chunk_name, format="wav")


        continue
 



   


In [ ]:
from pydub import AudioSegment
import librosa


test_dir = "/Users/michaelnguyen/Projects/environments/audio_segmentation/TEEEEST.wav"

audio_data, sampling_rate = librosa.load(test_dir)

print(type(audio_data), type(sampling_rate))

import matplotlib.pyplot as plt
import librosa.display
plt.figure(figsize=(14, 5))
librosa.display.waveplot(audio_data, sr=sampling_rate)